<a href="https://colab.research.google.com/github/shashikantnagare/openCV-Objects-Detection-Projects/blob/main/Train_YoloV3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Connect google drive**

In [3]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Sun Dec 20 18:18:54 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
 007.jpg
 1530700874197..jpg
 1582714853472..jpg
'2019 Calendar.gsheet'
 achievements.jpg
'achievements.jpg (1064×1064)'
'Annual budget.gsheet'
'AS CORPO_Process-flow.png'
'Assignment 2.. (1).ipynb'
'Assignment2(EDA $ Visualization) (1).ipynb'
'Assignment2(EDA $ Visualization).ipynb'
'Assignment 2...ipynb'
 BM_data.csv.gsheet
 Classroom
'clustering assignment (1).ipynb'
'clustering assignment.ipynb'
'Colab Notebooks'
'Contact Information.gform'
'customer-segmentation-dataset (1).zip'
 customer-segmentation-dataset.zip
'DecisiontreAssignment_skillathon-Copy1 (1).ipynb'
'DecisiontreAssignment_skillathon-Copy1 (2).ipynb'
 DecisiontreAssignment_skillathon-Copy1.ipynb
 DecisiontreAssignment_skillathon-Copy1.py
'Decisiontree,KNN,Randonforest Assingnment skillathon (1).ipynb'
'Decisiontree,KNN,Randonforest Assingnment skillathon (1).py'
'Decisiontree,KNN,Randonfor

**1) Clone the Darknet**



In [5]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14645, done.
remote: Total 14645 (delta 0), reused 0 (delta 0), pack-reused 14645
Receiving objects: 100% (14645/14645), 13.22 MiB | 24.47 MiB/s, done.
Resolving deltas: 100% (9970/9970), done.


**2) Compile Darknet using Nvidia GPU**


In [6]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [7]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [8]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [9]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [10]:
!echo "Koala" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [11]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-12-20 18:20:04--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  11.2MB/s    in 14s     

2020-12-20 18:20:18 (11.1 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [12]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/Adachi_20170906151015.txt  
  inflating: data/obj/Adachi_20170906151017.jpg  
  inflating: data/obj/Adachi_20170906151017.txt  
  inflating: data/obj/Adachi_20170906151020.jpg  
  inflating: data/obj/Adachi_20170906151020.txt  
  inflating: data/obj/Adachi_20170906151108.jpg  
  inflating: data/obj/Adachi_20170906151108.txt  
  inflating: data/obj/Adachi_20170906151123.jpg  
  inflating: data/obj/Adachi_20170906151123.txt  
  inflating: data/obj/Adachi_20170906151126.jpg  
  inflating: data/obj/Adachi_20170906151126.txt  
  inflating: data/obj/Adachi_20170906151132.jpg  
  inflating: data/obj/Adachi_20170906151132.txt  
  inflating: data/obj/Adachi_20170906151137.jpg  
  inflating: data/obj/Adachi_20170906151137.txt  
  inflating: data/obj/Adachi_20170906151144.jpg  
  inflating: data/obj/Adachi_20170906151144.txt  
  inflating: data/obj/Adachi_20170906151235.jpg  
  inflating: data/obj/Adachi_20170906151235.txt  
  inflating: 

In [13]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

15 0.457500 0.839167 0.051667 0.255000

['15', '0.457500', '0.839167', '0.051667', '0.255000']
0 data/obj/Adachi_20170906151144.txt
0 0.457500 0.839167 0.051667 0.255000
15 0.405000 0.589167 0.183333 0.128333

['15', '0.405000', '0.589167', '0.183333', '0.128333']
1 data/obj/Adachi_20170906102859.txt
0 0.405000 0.589167 0.183333 0.128333
15 0.515000 0.799167 0.110000 0.245000

['15', '0.515000', '0.799167', '0.110000', '0.245000']
2 data/obj/Adachi_20170906102924.txt
0 0.515000 0.799167 0.110000 0.245000
15 0.711667 0.745000 0.206667 0.316667

['15', '0.711667', '0.745000', '0.206667', '0.316667']
3 data/obj/Adachi_20170906151407.txt
0 0.711667 0.745000 0.206667 0.316667
15 0.196667 0.860000 0.100000 0.253333

['15', '0.196667', '0.860000', '0.100000', '0.253333']
4 data/obj/Adachi_20170906093849.txt
0 0.196667 0.860000 0.100000 0.253333
15 0.664167 0.608333 0.108333 0.206667

['15', '0.664167', '0.608333', '0.108333', '0.206667']
5 data/obj/Adachi_20170906105131.txt
0 0.664167 0.60833

In [14]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/Adachi_20170906094007.jpg', 'data/obj/Adachi_20170906150515.jpg', 'data/obj/Adachi_20170906105131.jpg', 'data/obj/Adachi_20170906102627.jpg', 'data/obj/Adachi_20170906105143.jpg', 'data/obj/Adachi_20170906093858.jpg', 'data/obj/Adachi_20170906102603.jpg', 'data/obj/Adachi_20170906102632.jpg', 'data/obj/Adachi_20170906144624.jpg', 'data/obj/Adachi_20170906103833.jpg', 'data/obj/Adachi_20170906094211.jpg', 'data/obj/Adachi_20170906102924.jpg', 'data/obj/Adachi_20170906150628.jpg', 'data/obj/Adachi_20170906143948.jpg', 'data/obj/Adachi_20170906093924.jpg', 'data/obj/Adachi_20170906094003.jpg', 'data/obj/Adachi_20170906093949.jpg', 'data/obj/Adachi_20170906093912.jpg', 'data/obj/Adachi_20170906150632.jpg', 'data/obj/Adachi_20170906150605.jpg', 'data/obj/Adachi_20170906105314.jpg', 'data/obj/Adachi_20170906151249.jpg', 'data/obj/Adachi_20170906102935.jpg', 'data/obj/Adachi_20170906151123.jpg', 'data/obj/Adachi_20170906102907.jpg', 'data/obj/Adachi_20170906102905.jpg', 'data/obj/A

In [15]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [16]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
 total_bbox = 8793, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.477844), count: 3, class_loss = 57.922604, iou_loss = 1.039440, total_loss = 58.962044 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.406852), count: 1, class_loss = 96.636101, iou_loss = 0.289116, total_loss = 96.925217 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 87.562325, iou_loss = 0.000000, total_loss = 87.562325 
 total_bbox = 8797, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 57.529476, iou_loss = 0.000000, total_loss = 57.529476 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.466775), count: 4, class_loss = 97.665688, iou_loss = 1.475151, total_loss = 99.140839 
